In [1]:
import numpy as np
import h5py
import CAS.dmrghandler.src.dmrghandler.hdf5_io as hdf5_io
import CAS.dmrghandler.src.dmrghandler.dmrg_processing as dp
from pathlib import Path
import openpyxl as px
import openpyxl.chart as px_chart
import pandas as pd

This notebook will collect the DMRG calculation data and then output an excel file with the data and plots (one sheet per calculation run), as well as csv files that contain the performance metric data (bond dimension, DMRG energy, time, discarded weights, etc.)

To run this notebook and collect the data, the following are needed:
- catalysis_instances.csv 
    
    This contains the metadata for the catalysist instances.
    Put into `catalyst_meta_data_path`
    ✓

- DMRG output datafolders (i.e. those that were output with the UUIDs)
    
    Put folder of these datafolders into `data_file_path`

- The run commands returned by the `prepare_calcs_*.py` files
    
    Put into `run_commands`
    ✓

- SLURM emails (.eml or other text format should work)
    
    The notebook will automatically collect the run times from the email subjects.
    Put folder of these emails into `slurm_emails_folder`
    ✓

- The FCIDUMP files
    
    Put the folder of these files into `fci_dump_folder`
    ✓

- Excel output save folder
    
    Put into `excel_storage_path`
    ✓
    
- name of output excel file
    
    Put into `excel_name`
    ✓

- Performance metrics folder location
    
    Put into `csv_storage_path`
    ✓

In [ ]:
"""
# fcidumps_df/fcidump.9_mo_n2-_{'Mo'_'def2-TZVP''default'_'def2-SVP'}_tbt_combined_df
sbatch ../02a44b7b-c68c-4065-952c-47ae128e59ff/submit_02a44b7b-c68c-4065-952c-47ae128e59ff.sh
# fcidumps_df/fcidump.31_4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../c3b07055-1d4e-4cb0-a6e6-8c4e513e5b91/submit_c3b07055-1d4e-4cb0-a6e6-8c4e513e5b91.sh
# fcidumps_df/fcidump.22_pc-_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../74bf2694-a8c7-4825-a0d3-7a1fcc454fbd/submit_74bf2694-a8c7-4825-a0d3-7a1fcc454fbd.sh
# fcidumps_df/fcidump.21_rc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../aeeb1e35-1b4d-4552-8225-99e62f85b9cf/submit_aeeb1e35-1b4d-4552-8225-99e62f85b9cf.sh
# fcidumps_df/fcidump.16_ts_1over4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../f83c2ca9-fe06-436a-ba52-ddf31af6f3f0/submit_f83c2ca9-fe06-436a-ba52-ddf31af6f3f0.sh
# fcidumps_df/fcidump.15_1_lut_react_{'Mo'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../9f05068f-1d88-4c99-9532-813c85861409/submit_9f05068f-1d88-4c99-9532-813c85861409.sh
# fcidumps_df/fcidump.12_mo_n2_{'Mo'_'def2-TZVP''default'_'def2-SVP'}_tbt_combined_df
sbatch ../cb489780-2e42-46af-b027-810a0d58ecba/submit_cb489780-2e42-46af-b027-810a0d58ecba.sh
"""

In [52]:
catalyst_meta_data_path=Path('../catalysis_instances.csv')
excel_name=Path('20240531_catalyst_mf_df.xlsx')
data_file_path = Path("../dmrg_data/data_storage_mf")

run_commands = \
"""
# up to 20 log10 hilbert space
#######################
# fcidumps_df/fcidump.29_2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../e1ee2d6a-490e-441b-8fd0-c8216e4be0c2/submit_e1ee2d6a-490e-441b-8fd0-c8216e4be0c2.sh
# fcidumps_df/fcidump.28_2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../e47a8e75-ca1a-4192-a0b0-c5a792552101/submit_e47a8e75-ca1a-4192-a0b0-c5a792552101.sh
# fcidumps_df/fcidump.27_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../702234e9-0fea-4fe2-bb6b-bf65477e8319/submit_702234e9-0fea-4fe2-bb6b-bf65477e8319.sh
# fcidumps_df/fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../deade281-228d-4224-82ce-d7048fb1c737/submit_deade281-228d-4224-82ce-d7048fb1c737.sh
# fcidumps_df/fcidump.25_ts_1over2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../da443680-cc8a-4aff-891f-c07a8c510943/submit_da443680-cc8a-4aff-891f-c07a8c510943.sh
# fcidumps_df/fcidump.24_ts_1over2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../6951fbf8-7776-4cb7-a50b-851d63cefd7b/submit_6951fbf8-7776-4cb7-a50b-851d63cefd7b.sh
# fcidumps_df/fcidump.23_pc-_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../0dd7dc42-6f07-4e70-9940-f9d544116d12/submit_0dd7dc42-6f07-4e70-9940-f9d544116d12.sh
# fcidumps_df/fcidump.20_rc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../3e27c5b4-54c2-412a-9b3c-6c481e891a44/submit_3e27c5b4-54c2-412a-9b3c-6c481e891a44.sh
# fcidumps_df/fcidump.19_I_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../55837739-9ce7-41c7-8d98-2e01104b68e3/submit_55837739-9ce7-41c7-8d98-2e01104b68e3.sh
# fcidumps_df/fcidump.18_I_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../e2854268-2e06-4836-9b35-8dc8bbf9c98c/submit_e2854268-2e06-4836-9b35-8dc8bbf9c98c.sh
# fcidumps_df/fcidump.17_ts_1over4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../005845bd-750e-45b8-9852-441546899608/submit_005845bd-750e-45b8-9852-441546899608.sh
# fcidumps_df/fcidump.14_1_lut_prod_{'Mo'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../a027bb58-94ef-4409-bbcc-5668d650f178/submit_a027bb58-94ef-4409-bbcc-5668d650f178.sh
# fcidumps_df/fcidump.13_1_lut_ts_{'Mo'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_combined_df
sbatch ../c3e942aa-1337-41f5-a68d-97a4da8fe15b/submit_c3e942aa-1337-41f5-a68d-97a4da8fe15b.sh
# fcidumps_df/fcidump.11_fecp2_s0_def2-tzvp_tbt_combined_df
sbatch ../5afb3915-24f3-473e-a37c-1f78531770a2/submit_5afb3915-24f3-473e-a37c-1f78531770a2.sh
# fcidumps_df/fcidump.10_fecp2__s0.5_def2-tzvp_tbt_combined_df
sbatch ../9c5bb7b2-e372-48db-86c5-656cbd7143f9/submit_9c5bb7b2-e372-48db-86c5-656cbd7143f9.sh


"""

In [53]:
slurm_emails_folder = Path('../slurm_emails/slurm_emails_mf')
fci_dump_folder  = Path("../fcidumps_directory/fcidumps_df")
excel_storage_path = Path("../excel_data_storage")
csv_storage_path = excel_storage_path/Path("20240531performance_metrics_store_mf_df")


In [54]:
catalyst_meta_data = pd.read_csv(catalyst_meta_data_path)

In [55]:

cc_wall_time_dict = {}
# Get all eml files in the folder
slurm_email_files = list(slurm_emails_folder.glob('*.eml'))
for email_file in slurm_email_files:
    with open(email_file, 'r') as f:
        use_next_line_subject = False
        obtained_run_time = False
        for line in f:
            if use_next_line_subject:
                line = line.strip()
                run_time = line.split(', ')[0].split(' ')[-1]
                # print(run_time)
                obtained_run_time = True
                use_next_line_subject = False
            if line.startswith('Subject: '):
                use_next_line_subject = True
                # print(line)
            elif line.startswith('   Command=../'):
                line = line.strip()
                uuid = line.split('/')[1]
                # print(uuid)
                assert obtained_run_time, 'Run time not yet obtained'
                cc_wall_time_dict[uuid] = run_time
                
print(len(cc_wall_time_dict))

67


In [56]:
#Load all fcidump file names
fcidump_files = [filename.name for filename in list(fci_dump_folder.glob('fcidump.*'))]
# fcidump_files


In [57]:
print(len(fcidump_files))

67


In [58]:

import io
buf = io.StringIO(run_commands)
orig_data_dict_list = []
count = 0
for line in buf.readlines():
    if line.startswith('### '):
        continue
    if line.startswith('# fcidumps_df/'):
        count += 1
        fcidump_name_temp = line.split('fcidumps_df/')[1].strip()
        dict_entry = {}
        # print(fcidump_name_temp)
        # if fcidump_name.endswith('}'):
        #     fcidump_test_string = fcidump_name.split('{')[0]
        # else:
        #     basis_end = fcidump_name.split('_')[-1]
        #     fcidump_test_string = fcidump_name.split('_'+basis_end)[0]+'_'
        fcidump_test_string = fcidump_name_temp.split('_')[0] 
        fcidump_tail = fcidump_name_temp[-15:-1]
        # print(fcidump_test_string)
        for fcidump_name in fcidump_files:
            if fcidump_name.startswith(fcidump_test_string):
                break
        dict_entry['fcidump'] = fcidump_name
        dict_entry['fcidump_test_string'] = fcidump_test_string
        dict_entry['fcidump_tail'] = fcidump_tail
    if line.startswith('sbatch ../'):
        calc_uuid = line.split('sbatch ../')[1].split('/')[0]
        # print(calc_uuid)
        dict_entry['Calc UUID'] = calc_uuid
        dict_entry['CC Wall Time'] = cc_wall_time_dict[calc_uuid]
        dict_entry['Attempt Result'] = ""
        orig_data_dict_list.append(dict_entry)
   

In [59]:
count

15

In [60]:
# catalyst_meta_data.columns

In [61]:
data_dict_list = []
for data_dict in orig_data_dict_list:
    new_data_dict = {}
    fcidump_test_string = data_dict['fcidump_test_string']
    fcidump_tail = data_dict['fcidump_tail']
    #Get metadata row
    data_row = catalyst_meta_data[(catalyst_meta_data['fcidump'].str.contains(fcidump_test_string))]
    new_data_dict['instance ID'] = data_row['instance ID'].values[0]
    new_data_dict['molecule'] = data_row['molecule'].values[0]
    new_data_dict['charge'] = data_row['charge'].values[0]
    new_data_dict['multiplicity'] = data_row['multiplicity'].values[0]
    new_data_dict['fcidump'] = data_dict['fcidump'] + fcidump_tail
    new_data_dict['log10_hilbert_space_size'] = data_row['log10_hilbert_space_size'].values[0]
    new_data_dict['Attempt Result'] = data_dict['Attempt Result']
    new_data_dict['Calc UUID'] = data_dict['Calc UUID']
    new_data_dict['CC Wall Time'] = data_dict['CC Wall Time']
    new_data_dict['fcidump_tail'] = fcidump_tail
    if 'Calc UUID Small BD' in data_dict:
        new_data_dict['Calc UUID Small BD'] = data_dict['Calc UUID Small BD']
        new_data_dict['CC Wall Time Small BD'] = data_dict['CC Wall Time Small BD']
    data_dict_list.append(new_data_dict)



    

In [62]:
print(len(data_dict_list))

15


In [63]:
# data_dict_list

In [64]:
wb = px.Workbook()

dp.setup_workbook(data_file_path=data_file_path,data_dict_list=data_dict_list,workbook=wb,csv_storage_path=csv_storage_path)

wb.save(excel_storage_path/excel_name)

Last loop included = 46
Last loop included = 51
Last loop included = 46
Last loop included = 53
Last loop included = 46
Last loop included = 32
Processed results available
Checking that processed results match raw results.
Last loop included = 3
Last loop included = 43
Processed results available
Checking that processed results match raw results.
Last loop included = 1
Last loop included = 44
Last loop included = 4
Last loop included = 19
Last loop included = 28
Last loop included = 49
Last loop included = 48


/Users/rick/planted_solutions/planted_solutions/venv/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
